Remove useless neurons

In [1]:
import os
import torch
import numpy as np
import gymnasium as gym
from tianshou.data import Collector, VectorReplayBuffer, Batch
from tianshou.utils.net.common import Net
from gymnasium.wrappers import TransformObservation
from tianshou.env import SubprocVectorEnv
from tianshou.policy import DQNPolicy
import pprint

weights = np.load('weights.npy')
hiddens = np.load('hidden_outputs.npy')
actions = np.load('actions.npy')
states = np.load('states.npy')
def make_env():
    env = gym.make("Blackjack-v1", natural=False, sab=False)
    env = TransformObservation(env, lambda obs: np.array(obs), observation_space=None)
    return env


DEVICE = "cuda:0"
env = make_env()
state_shape = 3
action_shape = env.action_space.shape or env.action_space.n

q_net = Net(state_shape, action_shape, hidden_sizes=[64] * 2, device=DEVICE).to(DEVICE)
optim = torch.optim.Adam(q_net.parameters(), lr=2.5e-4)
policy = DQNPolicy(
    model=q_net,
    optim=optim,
    action_space=env.action_space,
    discount_factor=0.99,
    estimation_step=3,
    target_update_freq=500,
).to(DEVICE)
policy.eval()
checkpoint_path = "/root/gym/rl_compexp/save/Blackjack2-DQN64/dqn_best.pth"
policy.load_state_dict(torch.load(checkpoint_path,map_location=DEVICE))
hidden_outputs = []
def hook_fn(module, input, output):
    # Assuming output is a tensor, detach and move to CPU
    hidden_outputs.append(output.detach().cpu().numpy())


policy.model.model.model[2].register_forward_hook(hook_fn)


In [2]:
neuron = 28
index = hiddens[:,neuron]>0

selected_states = states[index]
print(selected_states.shape)
selected_actions = actions[index]
selected_hidden_outputs = hiddens[index]
concept = "((((P18 OR P20) OR P19) OR P17) OR P21)"
# for i in range(10):
#     state = selected_states[i]
#     act0 = policy(Batch(obs=state.reshape(1, -1), info="")).act.item()
# print(f"origin action: {act0}, origin state: {state}, origin hidden:{hidden_outputs[-1][0,neuron]}")
origin_state = selected_states[1]
act0 = policy(Batch(obs=origin_state.reshape(1, -1), info="")).act.item()
print(f"origin action: {act0}, origin state: {origin_state}, origin hidden:{hidden_outputs[-1][0,neuron]}")

purt_state = selected_states[1].copy()
purt_state[0] = 14
act1 = policy(Batch(obs=purt_state.reshape(1, -1), info="")).act.item()
print(f"purt action: {act1}, purt state: {purt_state}, purt hidden:{hidden_outputs[-1][0,neuron]}")

(4462, 3)
origin action: 0, origin state: [20  9  0], origin hidden:2.0437397956848145
purt action: 1, purt state: [14  9  0], purt hidden:-1.0301517248153687


In [3]:
neuron = 13
index = hiddens[:,neuron]>0

selected_states = states[index]
print(selected_states.shape)
selected_actions = actions[index]
selected_hidden_outputs = hiddens[index]
concept = "P6-P10"
# for i in range(10):
#     state = selected_states[i]
#     act0 = policy(Batch(obs=state.reshape(1, -1), info="")).act.item()
#     print(f"origin action: {act0}, origin state: {state}, origin hidden:{hidden_outputs[-1][0,neuron]}")
i=0
origin_state = selected_states[i]
act0 = policy(Batch(obs=origin_state.reshape(1, -1), info="")).act.item()
print(f"origin action: {act0}, origin state: {origin_state}, origin hidden:{hidden_outputs[-1][0,neuron]}")

purt_state = selected_states[i].copy()
purt_state[0] = 17
act1 = policy(Batch(obs=purt_state.reshape(1, -1), info="")).act.item()
print(f"purt action: {act1}, purt state: {purt_state}, purt hidden:{hidden_outputs[-1][0,neuron]}")

(1349, 3)
origin action: 1, origin state: [6 9 0], origin hidden:2.0420312881469727
purt action: 0, purt state: [17  9  0], purt hidden:-6.891425132751465


In [4]:
neuron = 17
index = hiddens[:,neuron]>0

selected_states = states[index]
print(selected_states.shape)
selected_actions = actions[index]
selected_hidden_outputs = hiddens[index]
concept = "D7-D10"
# for i in range(10):
#     state = selected_states[i]
#     act0 = policy(Batch(obs=state.reshape(1, -1), info="")).act.item()
#     print(f"origin action: {act0}, origin state: {state}, origin hidden:{hidden_outputs[-1][0,neuron]}")
i=1
origin_state = selected_states[i]
act0 = policy(Batch(obs=origin_state.reshape(1, -1), info="")).act.item()
print(f"origin action: {act0}, origin state: {origin_state}, origin hidden:{hidden_outputs[-1][0,neuron]}")

purt_state = selected_states[i].copy()
purt_state[1] = 5
act1 = policy(Batch(obs=purt_state.reshape(1, -1), info="")).act.item()
print(f"purt action: {act1}, purt state: {purt_state}, purt hidden:{hidden_outputs[-1][0,neuron]}")

(5828, 3)
origin action: 1, origin state: [15  9  0], origin hidden:1.069049596786499
purt action: 0, purt state: [15  5  0], purt hidden:-0.9205737113952637


In [ ]:
# 获取排序后的索引
sorted_indices = np.argsort(weights, axis=0)[::-1]

# 根据索引获取排序后的值
sorted_weights = np.take_along_axis(weights, sorted_indices, axis=0)

print(sorted_indices)
print(sorted_weights)

In [1]:
import pandas as pd

mean_iou = {}
i = 5
res = pd.read_csv(f"result_{i}.csv")
res = res[res["iou"] > 0]
res = res.sort_values(by="iou", ascending=False)
mean_iou[i] = res["iou"].mean()
res.to_csv(f"result_{i}_parsed.csv", index=False)

find 4 outputs top connections

In [ ]:
import pandas as pd

i = 5
N = 10
csv = pd.read_csv(f"result_{i}_parsed.csv")
output_file = f"output_{i}.txt"
neurons = []


def find_unique_elements(arr1, arr2):
    set1, set2,  = set(arr1), set(arr2)
    unique1 = set1 - set2 
    unique2 = set2 - set1 


    return unique1, unique2


for label in ["w_stick","w_hit"]:
    sorted_csv = csv.sort_values(by=label, ascending=False)
    top_neurons = sorted_csv.head(N)["neuron"].tolist()
    neurons.append(top_neurons)

unique_elements = find_unique_elements(neurons[0], neurons[1])
print(unique_elements, "\n")

with open(output_file, "w") as f:
    for i, label in enumerate(["w_stick", "w_hit"]):
        csv = csv.sort_values(by=label, ascending=False)
        top_neurons = csv.head(N)[["neuron", label]].to_records(index=False)
        print(f"\n============\n{label} top {N}:", file=f)
        for neuron, value in top_neurons:
            if neuron in unique_elements[i]:
                print(f"* {neuron} {value}", file=f)  # Special mark for unique elements
                print(f"* {neuron} {value}")
            else:
                print(f"  {neuron} {value}", file=f)
                print(f"  {neuron} {value}")

find act & inp pair

In [ ]:
import numpy as np

act = np.load(f"../save/LunarLander-{NET_NAME}/actions.npy")
obs = np.load("../save/lunar.npy")

act_list = []
act_idx_list = []
max_indices = np.argmax(act, axis=1)
print(max_indices)
print(max_indices.shape)
for i in range(4):
    max_row_index = np.where(max_indices == i)[0]
    print(max_row_index.shape)
    # max_row = act[max_row_index]
    act_idx_list.append(max_row_index)
    act_list.append(obs[max_row_index])

# np.save(f'../save/LunarLander-{NET_NAME}/max_indices.npy', max_indices)

## analyze specific neuron(DQN)

In [ ]:
from analyze import *

ckpt_path = "../save/ckpt-mlp1024-260.pth"
data_path = "../save/lunar.npy"
print("Load model and dataset")
model, dataset = load_for_analysis(ckpt_path, data_path)
weights = model.fc3.weight.t().detach().cpu().numpy()

print("Extract features")
inputs, features, outputs = extract_feature(model, dataset)
outputs = np.array(outputs)
if not os.path.exists(settings.RESULT):
    # 如果不存在，创建路径
    os.makedirs(settings.RESULT)
np.save(os.path.join(settings.RESULT, f"actions.npy"), outputs)
print("Computing quantiles")
activations = quantile_features(features)
print("Generating atomic masks")
feat_masks = gen_feat_mask(inputs)
print(feat_masks.shape)

[x, y, Vx, Vy, angle, angular_v, l, r]

y_near_ground: <0.4

Vy_low: -0.5 ~ 0

In [ ]:
def intersect(x: np.ndarray, y: np.ndarray):
    return np.array(list(set(x).intersection(set(y))))


vy_low = np.where(feat_masks[:, 7] > 0)[0]
y_near_ground = np.where(feat_masks[:, 4] > 0)[0]

vy_low_and_y_near_ground = intersect(vy_low, y_near_ground)

neuron_act = np.where(activations[:, 630] > 0)[0]
res = intersect(neuron_act, act_idx_list[0])
res = intersect(res, vy_low_and_y_near_ground)
print(res.shape)
print(act[res[10]])
print(obs[res[10]])
print(features[res[10]][630])
hook = HookLayer()
hook.hook_layer(layer=model.act2)
obs_attk = obs[res[10]]
obs_attk[2] = 1.0
print(obs_attk)
act_attk = model(torch.tensor(obs_attk, dtype=torch.float32).to(settings.DEVICE))
print(hook.features_blobs)

In [ ]:
print(hook.features_blobs[0][630])
print(act_attk)

## analyze attack (PPO)


In [ ]:
from PPO import get_PPO
import settings
from analyze import *

ckpt_path = "../save/LunarLander-PPO1024/ppo.pth"
data_path = "../save/lunar.npy"
print("Load model and dataset")
inputs, features, outputs, weight, policy = get_PPO(ckpt_path, data_path)


if not os.path.exists(settings.RESULT):
    # 如果不存在，创建路径
    os.makedirs(settings.RESULT)
np.save(os.path.join(settings.RESULT, f"actions.npy"), outputs)
print("Computing quantiles")
activations = quantile_features(features)
print("Generating atomic masks")
feat_masks = gen_feat_mask(inputs)
print(feat_masks.shape)

In [ ]:
import tianshou


def intersect(x: np.ndarray, y: np.ndarray):
    return np.array(list(set(x).intersection(set(y))))


not_l_leg = np.where(feat_masks[:, 14] == 0)[0]
r_leg = np.where(feat_masks[:, 15] > 0)[0]
x_in_centor = np.where(feat_masks[:, 0] > 0)[0]

inter = intersect(not_l_leg, r_leg)
inter = intersect(x_in_centor, inter)

neuron_act = np.where(activations[:, 212] > 0)[0]
res = intersect(neuron_act, act_idx_list[2])
res = intersect(res, inter)
print(res.shape)
print(act[res[10]])
print(obs[res[10]])
print(features[res[10]][212])
hook = HookLayer()
hook.hook_layer(policy.actor.preprocess.model.model[2])
obs_attk = obs[res[10]]
obs_attk[0] = 0.9
obs_attk[6] = 1.0
obs_attk = obs_attk.reshape(1, -1)
print(obs_attk)
print(obs_attk.shape)
act_attk = policy(tianshou.data.Batch(obs=obs_attk, info=""))
print(hook.features_blobs[0][0][212])
print(act_attk)